# LVM Data Simulator tutorial: 2D simulations


**This is a tutorial on how run the simulator to produce 2D Raw spectra in the same format as they will be obtained with LVM. This is developed mainly for DRP tests.**
- How to install the simulator and how to prepare a custom simulation is shown in [tutorial #1](https://github.com/sdss/lvmdatasimulator/blob/main/examples/LVM_data_simulator_tutorial1.ipynb)
- More details on the setup of LVM instrumentation and on the structure of the outputs of the simulator are given in [tutorial #2](https://github.com/sdss/lvmdatasimulator/blob/main/examples/LVM_data_simulator_tutorial2.ipynb)
- More realistic and useful simulations are considered in [tutorial #3](https://github.com/sdss/lvmdatasimulator/blob/main/examples/LVM_data_simulator_tutorial3.ipynb) - please go there if you have already installed and configured the package and want to learn how to simulate the source field in more details.
- If you want to learn how to use the code as exposure time calculator (ETC) - please go there [tutorial #4](https://github.com/sdss/lvmdatasimulator/blob/main/examples/LVM_data_simulator_tutorial4.ipynb).

_Last update: 2023/02/27_


Producing of 2D Raw LVM spectra is very similar to 1D case, but you need to use the `Simulator2D` class instead of the `Simulator`. Modelling of the source field, initialization of the instrument (telescope + spectrograph) properties and of the observational conditions are performed in the the same was as for 1D case (see tutorials 1--3 for details).

In [1]:
from lvmdatasimulator.simulator2d import Simulator2D
from lvmdatasimulator.field import LVMField
from lvmdatasimulator.observation import Observation
from lvmdatasimulator.telescope import LVM160
from lvmdatasimulator.instrument import LVMSpectrograph
from lvmdatasimulator.fibers import FiberBundle

import astropy.units as u
from matplotlib import pyplot as plt
import os

Go to the directory where the results will be saved

In [2]:
os.chdir("/Users/mors/Science/LVM/Sim2D/")

Below we initialize the properties of the telescope, spectrograph, fiber bundles, observational conditions, source field. We skip the details here (see tutorials 1-3).
We will create relatively complex source field containing DIG and two expanding bubbles. We are going to simulate the spectra for the entire fiber bundle.
__N.B.: The simulation below takes 40-60 minutes. To decrease this time, you can simplify the source field (e.g., exclude the bubbles) or/and limit the number of rings in the `FiberBundle`.__

In [3]:
tel = LVM160()
spec = LVMSpectrograph()

bundle = FiberBundle(bundle_name='full')  # add nrings=XX in order to limit the simulations by XX rings only

ra = 12.34 # u.degree
dec = -20.35 # u.degree
fov_size = 35 # u.arcmin
fov_pixel = 5 # u.arcsec
distance = 50 # * u.kpc
turbulent_sigma = 20 #* u.km / u.s

exptimes = [900] # in seconds
obs = Observation(ra=ra, dec=dec, unit_ra=u.deg, unit_dec=u.deg, exptimes=exptimes)

[INFO]: Using the full hexagon pattern


In [4]:
name = 'LVM_2D_full'
my_lvmfield = LVMField(ra=ra, dec=dec, size=fov_size, pxsize=fov_pixel, name=name,
                       ism_params={'distance': distance, 'turbulent_sigma': turbulent_sigma, 'vel_amplitude': 250})
# my_lvmfield.generate_gaia_stars(gmag_limit=15)
# dig = {"type": 'DIG', 'max_brightness': 5e-16, 'perturb_amplitude': 0.2, 'perturb_scale': 500 * u.pc}
# bubble1 = {'type': 'Bubble', 'max_brightness': 2e-14, 'radius': 40, 'expansion_velocity':50, 'thickness':0.2, 'sys_velocity': 40,
#           'model_params': {'Z': 0.8, 'qH': 51., 'nH': 10}, 'model_type': 'cloudy', 'offset_X': 0, 'offset_Y':-40}
# bubble2 = {'type': 'Bubble', 'max_brightness': 1e-15, 'radius': 70, 'expansion_velocity':25, 'thickness':0.3, 'sys_velocity': -30,
#           'model_params': {'Z': 0.8, 'qH': 50., 'nH': 10}, 'model_type': 'cloudy', 'offset_X': -40, 'offset_Y':90}
#
# my_lvmfield.add_nebulae([dig, bubble1, bubble2])

In [5]:
my_lvmfield.show(cmap=plt.cm.Greys, percentile=96, fibers=bundle.fibers_science)

[WARNING]: Bundle center coords are not defined, using the coords of the field.


KeyboardInterrupt: 

We created the source field and initialized all other required objects. Now we put everything into 2D simulator defined by the class `Simulator2D`. Its initialization is the same as for `Simulator` in 1D case, but the available methods are different

In [5]:
sim = Simulator2D(my_lvmfield, obs, spec, bundle, tel)

Now we create the 2D Raw spectra for all fibers in the FOV. The input spectra will be convolved with the instrumental LSF and PSF specified for all 3 channels and cameras and varying with the position on CCD and the wavelength. This is computationally expensive and takes ~30 min per channel and camera.

In [8]:
sim.simulate_science()

[INFO]: Recovering target spectra for 1801 fibers.
[INFO]: Start extracting nebular spectra
[INFO]: Reading the atmospheric extinction from file.
[INFO]: Simulating the sky emission 2 days from new moon.
[INFO]: Using sky file: /Users/mors/Science/PyWorkspace/lvmdatasimulator/data/sky/LVM_LVM160_SKY_2.dat
[INFO]: Resample sky emission to instrument wavelength solution.
[INFO]: Generating standard stars
[INFO]: Standard star 1 with Teff 6250, Gmag 8.50 added.
[INFO]: Standard star 2 with Teff 6000, Gmag 8.80 added.
[INFO]: Standard star 3 with Teff 6500, Gmag 5.80 added.
[INFO]: Standard star 4 with Teff 6750, Gmag 7.90 added.
[INFO]: Standard star 5 with Teff 6750, Gmag 6.20 added.
[INFO]: Standard star 6 with Teff 7000, Gmag 5.20 added.
[INFO]: Standard star 7 with Teff 6250, Gmag 6.90 added.
[INFO]: Standard star 8 with Teff 5750, Gmag 7.30 added.
[INFO]: Standard star 9 with Teff 6500, Gmag 8.00 added.
[INFO]: Standard star 10 with Teff 6750, Gmag 6.30 added.
[INFO]: Standard star 1

[INFO]: Saving science exposures with 900s exposures and 10s of exposure for each standard star
[INFO]: Project the spectra of camera #1 and blue channel onto CCD
100%|██████████| 648/648 [27:31<00:00,  2.55s/it]
[INFO]: Done for camera #1 and blue branch
[INFO]: Project the spectra of camera #1 and red channel onto CCD
100%|██████████| 648/648 [34:48<00:00,  3.22s/it]
[INFO]: Done for camera #1 and red branch
[INFO]: Project the spectra of camera #1 and ir channel onto CCD
100%|██████████| 648/648 [34:37<00:00,  3.21s/it] 
[INFO]: Done for camera #1 and ir branch


We can also create 2D output for all the calibrations that we need (flat, arcs, bias)

In [6]:
sim.create_flat()

[INFO]: Creating flat fields...
[INFO]: Saving flat exposures with exptime 10s
[INFO]: Project the spectra of camera #1 and blue channel onto CCD
100%|██████████| 648/648 [00:03<00:00, 201.73it/s]


TypeError: 'NoneType' object is not subscriptable

In [1]:
from lvmdatasimulator.simulator2d import Simulator2D
from lvmdatasimulator.field import LVMField
from lvmdatasimulator.observation import Observation
from lvmdatasimulator.telescope import LVM160
from lvmdatasimulator.instrument import LVMSpectrograph
from lvmdatasimulator.fibers import FiberBundle

import astropy.units as u
from matplotlib import pyplot as plt
import os

os.chdir("/Users/mors/Science/LVM/Sim2D/")

name = 'LVM_2D_full'
tel = LVM160()
spec = LVMSpectrograph()

bundle = FiberBundle(bundle_name='full')  # add nrings=XX in order to limit the simulations by XX rings only

ra = 12.34 # u.degree
dec = -20.35 # u.degree
fov_size = 35 # u.arcmin
fov_pixel = 5 # u.arcsec
distance = 50 # * u.kpc
turbulent_sigma = 20 #* u.km / u.s

exptimes = [900] # in seconds
obs = Observation(ra=ra, dec=dec, unit_ra=u.deg, unit_dec=u.deg, exptimes=exptimes)
my_lvmfield = LVMField(ra=ra, dec=dec, size=fov_size, pxsize=fov_pixel, name=name,
                       ism_params={'distance': distance, 'turbulent_sigma': turbulent_sigma, 'vel_amplitude': 250})

sim = Simulator2D(my_lvmfield, obs, spec, bundle, tel)
sim.create_flat()

[INFO]: Using the full hexagon pattern
[INFO]: Creating flat fields...
[INFO]: Saving flat exposures with exptime 10s
[INFO]: Project the spectra of camera #1 and blue channel onto CCD
100%|██████████| 646/646 [06:44<00:00,  1.60it/s]
[INFO]: Done for camera #1 and blue branch
[INFO]: Project the spectra of camera #2 and blue channel onto CCD
 10%|▉         | 64/647 [00:45<06:50,  1.42it/s] 


KeyboardInterrupt: 